## Camera calibration

Place the micron scale calibration pattern on the printing surface.  Use Guvcview to adjust the USB microscope focus and magnification to give a field of view of roughly 10 mm by 10 mm.  Exit the Guvcview application before running this calibration script.

In [ ]:
import cv2 as cv
import numpy as np

The code cell below captures an image from the USB microscope video stream. This image is named "calibration.png" and stored in the current working directory.  The image resolution is set here at 1600x1200.  If it is desired to alter this resolution, check that the new resolution is supported by the camera.  Supported resolutions can be found from a drop down list on Guvcview.  Ignore the harmless warning "GStreamer warning: Cannot query video position."  This warning is because we are capturing from a live video stream, not a video file.

In [ ]:
video_capture = cv.VideoCapture(0)
# Check success
if not video_capture.isOpened():
    raise Exception("Could not open video device")
# Set properties. Each returns === True on success (i.e. correct resolution)
video_capture.set(cv.CAP_PROP_FRAME_WIDTH, 1600)
video_capture.set(cv.CAP_PROP_FRAME_HEIGHT, 1200)
# Read picture. ret === True on success
ret, frame = video_capture.read()
cv.imwrite('calibration.png',frame)
cv.imwrite('calibrationCopy.png',frame)
# Close device
video_capture.release()

The code cell below opens up the captured image in a new window. Take a look and make sure the image is acceptable, then close the window.  

If desired, the code cell above can be run again to capture a new image.  The new image file will overwrite the old one.

In [ ]:
img = cv.imread('calibration.png')
winname = 'Calibration Image' # set name of window
cv.imshow(winname, img)
while cv.getWindowProperty(winname, cv.WND_PROP_VISIBLE) > 0:
    if cv.waitKey(100) > 0:
        break
cv.destroyAllWindows()

To calibrate the camera, you will click two points on the image separated by a known distance on the calibration pattern.  The distance between these two points is the calibration scale.  Run the code cell below to set the desired calibration scale in units of millimeters.  The calibration scale should be in the range of 1 mm to 10 mm.

In [ ]:
scale = 0 # zero initial value outide acceptable range
while scale>10 or scale<1:# prompt for input between 1 and 10 mm
    scale=float(input())

Next, manually click on pairs of points on the calibration image that are separated by the desired scale.  You will be prompted whether to keep or reject the points by pressing Y or N. To obtain a better estimate, this process will be repeated five times, and the results averaged.

In [ ]:
points = [] # empty lists for adding points marked on image

# define function for capturing pair of calibration point coordinates
def click_event(event, x, y, flags, param):
    if event == cv.EVENT_LBUTTONDOWN:
        if len(tpoints)<2:
            points.append((x, y))
            tpoints.append((x, y))
            cv.circle(img, (x, y), 3, (0, 255, 0), -1)
            cv.putText(img, str(x) + ',' + str(y), (x+10, y+10), cv.FONT_HERSHEY_COMPLEX,
                           0.75, (0, 255, 0), 1)
            cv.imshow(winname, img)
        else:
            cv.putText(img, 'Close this window.', (100, 200), cv.FONT_HERSHEY_COMPLEX,
                           0.75, (0, 255, 0), 1)
            cv.imshow(winname, img)
    return

# repeat collecting pairs of calibration points 5 times
for i in range(5):
    tpoints = []
    cv.putText(img, 'Click two points separated by ' + str(scale) + ' mm', (100, 100), cv.FONT_HERSHEY_COMPLEX,
                           0.75, (0, 255, 0), 1)
    cv.imshow(winname, img)
    cv.setMouseCallback(winname, click_event)
    while cv.getWindowProperty(winname, cv.WND_PROP_VISIBLE) > 0:
        if cv.waitKey(100) > 0:
            break
    cv.destroyAllWindows()
    img = cv.imread('calibrationCopy.png') # overwrite modified image with old one
    cv.imwrite('calibration.png',img)

# convert list of cellected points into numpy array
pointsnp = np.asarray(points)

In [ ]:
pixdist = np.zeros(5) # create empty array for storing distance between points in pixels
n = 0
for i in range(0,len(pointsnp),2):
    pixdist[n] = ((pointsnp[i,0]-pointsnp[i+1,0])**2+(pointsnp[i,1]-pointsnp[i+1,1])**2)**0.5
    n+=1

calscale = 1000*(scale/pixdist) # calibration scale in microns per pixel

np.savetxt('calscale.csv',np.array([np.mean(calscale)]))

print('Measured distanced (in pixels) between 5 pairs of points selected.')
print(pixdist)

In [ ]:
print('Measured pixels per mm:')
print('     mean =',np.mean(pixdist/scale))
print('     maximum =',np.amax(pixdist/scale))
print('     minimum =',np.amin(pixdist/scale))
print('     standard deviation =',np.std(pixdist/scale,ddof=1))
print(' ')
print('Mean value gives a scale of {:.1f} microns per pixel.'.format(np.mean(calscale)))

If these values are acceptable, the calscale variable (mean measured microns per pixel) is saved in the current directory as a csv file named 'calscale.csv'. The calibration scale can be imported from this file for taking position measurements from images.

If there are any outliers in the measured calbration values, repeat the calibration process by re-running this notebook.  The calscale.csv file will be overwritten each time this Jupyter notebook is run.